In [1]:
import psycopg2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy
#import plotly
#import plotly.graph_objs as go

#import plotly.plotly as py
#import cufflinks as cf


#from sklearn.metrics import r2_score

import rpy2
print(rpy2.__version__)

#import warnings
#from collections import Counter

2.9.4


In [2]:

try:
    connection = psycopg2.connect(user="sesyr",
                                  password="SeSyR54",
                                  host="51.254.68.36",
                                  port="5432",
                                  database="donnees_trafic")
    cursor = connection.cursor()

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")
    
    postgreSQL_select_Query = """
    WITH metadata AS (
          SELECT ist.macrosection,
         ist.code_centrale,
         cnl.code_canal,
         sc.sens,
         cc.id_nature_mesure, cc.libelle
        FROM rdv.info_section_troncon ist
        JOIN rdv.sens_canal sc ON sc.sens::text = ist.sens::text
    JOIN myrabel.centrale c ON c.code_centrale = ist.code_centrale::bpchar
    JOIN myrabel.canal cnl ON cnl.id_centrale = c.id_centrale AND cnl.code_canal::text = sc.canal::text
    JOIN myrabel.cal_canal cc ON cc.id_canal = cnl.id_canal AND cc.libelle ~~ '%QT_360 %'::text
    
    ), metadatavt AS (
          SELECT distinct ist.macrosection,
         ist.code_centrale,
         cnl.code_canal,
         sc.sens,
         cc.id_nature_mesure, cc.libelle
        FROM rdv.info_section_troncon ist
        JOIN rdv.sens_canal sc ON sc.sens::text = ist.sens::text
    JOIN myrabel.centrale c ON c.code_centrale = ist.code_centrale::bpchar
    JOIN myrabel.canal cnl ON cnl.id_centrale = c.id_centrale AND cnl.code_canal::text = sc.canal::text
    JOIN myrabel.cal_canal cc ON cc.id_canal = cnl.id_canal AND cc.libelle ~~ '%VT_360 %'::text
    )
    
    SELECT 
       
       m.macrosection,
       m.sens,
       m.code_centrale,
       m.code_canal, 
       
        
       ts,
       dq.valeur AS qt,
       dv.valeur AS vt
       
       
      FROM metadata  m
      JOIN metadatavt v
      ON m.macrosection = v.macrosection AND m.sens = v.sens AND m.code_centrale = v.code_centrale AND  m.code_canal=  v.code_canal
      
      CROSS JOIN generate_series('2017-05-01' , '2017-05-07','00:06:00'::interval) AS  ts(ts)
      
      LEFT JOIN myrabel.mesure_2 dq    
      ON dq.id_nature_mesure = m.id_nature_mesure
      AND dq.date_mesure = ts 
      
        LEFT JOIN myrabel.mesure_2 dv    
      ON dv.id_nature_mesure = v.id_nature_mesure
      AND dv.date_mesure = ts
      
      
        --WHERE(  dv.date_mesure IS NULL OR dq.date_mesure IS NULL OR dv.date_mesure = dq.date_mesure )
      
     -- WHERE dq.date_mesure IS NOT  NULL AND dv.date_mesure IS NULL  --OR dq.date_mesure IS NULL
      
      """
    

    cursor.execute(postgreSQL_select_Query)
    print("Selecting rows from mobile table using cursor.fetchall")
    records = cursor.fetchall() 
   
   # print("Print each row and it's columns values")
   #for row in records:
       #   print("Macrosection = ", row[0])
        #  print("sens = ", row[1])
         # print("code centrale   = ", row[2])
          #print(" code canal = ", row[3])
        #  print("date_mesure = ", row[4])
         # print("qt_360 = ", row[5]) 
          #print("Vt_360 = ", row[6],"\n" )
            
            
except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
        
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")            

You are connected to -  ('PostgreSQL 10.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-28), 64-bit',) 

Selecting rows from mobile table using cursor.fetchall
PostgreSQL connection is closed


In [3]:
df = pd.DataFrame(records, columns=['Macrosection','sens','code_centrale','code_canal','date','qt_360','vt_360'])

#strip code canal which is a char of 3 character with whitespace

df['code_canal'] = df['code_canal'].str.strip()

# add a column that its default value is np.nan

# df.missing

#sorting dates ?
#df = df.sort_values('date')

# replace zeros also with NAN

df = df.replace(0,np.NaN) # np.NaN ca va marcher , np.nan doesnt work anymore !!

# Descriptive Statistics

df.head()

,Macrosection,sens,code_centrale,code_canal,date,qt_360,vt_360
0,I11,SENS_1,MZE54.3,0,2017-05-01 00:00:00+02:00,33.0,97.0
1,I11,SENS_1,MZE54.3,0,2017-05-01 00:06:00+02:00,26.0,95.0
2,I11,SENS_1,MZE54.3,0,2017-05-01 00:12:00+02:00,36.0,98.0
3,I11,SENS_1,MZE54.3,0,2017-05-01 00:18:00+02:00,29.0,96.0
4,I11,SENS_1,MZE54.3,0,2017-05-01 00:24:00+02:00,37.0,94.0


In [4]:
DataFrameDict=df.groupby(['code_canal','code_centrale'])
    
#print to s
for key, item in DataFrameDict:
    print(DataFrameDict.get_group(key).count(), "\n\n")
    #DataFrameDict.get_group(key).to_csv(r'C:\Users\hp\Desktop\Stage IOT TS\devellopement\grouping_by_codes'+ str(key) +'.csv')
    


Macrosection     1441
sens             1441
code_centrale    1441
code_canal       1441
date             1441
qt_360           1441
vt_360           1441
dtype: int64 


Macrosection     1441
sens             1441
code_centrale    1441
code_canal       1441
date             1441
qt_360           1441
vt_360           1441
dtype: int64 


Macrosection     1441
sens             1441
code_centrale    1441
code_canal       1441
date             1441
qt_360           1441
vt_360           1441
dtype: int64 


Macrosection     1441
sens             1441
code_centrale    1441
code_canal       1441
date             1441
qt_360           1440
vt_360           1440
dtype: int64 


Macrosection     1441
sens             1441
code_centrale    1441
code_canal       1441
date             1441
qt_360           1440
vt_360           1440
dtype: int64 


Macrosection     1441
sens             1441
code_centrale    1441
code_canal       1441
date             1441
qt_360           1440
vt_360           1

In [5]:
#store data in alias dictionary

DataGp = {elem : pd.DataFrame for elem,it in DataFrameDict}

for key in DataGp.keys():
        print(key,end ="  ")
        DataGp[key] = DataFrameDict.get_group(key)
#27 groups        

('0', 'MZE54.3')  ('0', 'MZE54.H')  ('0', 'SITE_03')  ('0', 'SITE_05')  ('0', 'SITE_08')  ('1', 'MZE54.3')  ('1', 'MZE54.4')  ('1', 'MZE54.H')  ('1', 'MZE54.K')  ('1', 'SITE_01')  ('1', 'SITE_03')  ('1', 'SITE_05')  ('1', 'SITE_08')  ('1', 'SITE_09')  ('2', 'MZE54.H')  ('2', 'SITE_03')  ('2', 'SITE_05')  ('2', 'SITE_08')  ('3', 'MZE54.H')  ('3', 'SITE_01')  ('3', 'SITE_03')  ('3', 'SITE_05')  ('3', 'SITE_08')  ('3', 'SITE_09')  ('5', 'SITE_01')  ('5', 'SITE_08')  ('5', 'SITE_09')  

In [6]:
from IPython.display import display

pd.options.display.max_columns = None

#  for object type (or categorical) columns only 
DataFrameDict.describe(include='O')

Macrosection                    sens                 \
                                count unique  top  freq count unique     top   
code_canal code_centrale                                                       
0          MZE54.3               1441      1  I11  1441  1441      1  SENS_1   
           MZE54.H               1441      1  I12  1441  1441      1  SENS_1   
           SITE_03               1441      1  I11  1441  1441      1  SENS_1   
           SITE_05               1441      1  I11  1441  1441      1  SENS_1   
           SITE_08               1441      1  I12  1441  1441      1  SENS_1   
1          MZE54.3               1441      1  I23  1441  1441      1  SENS_2   
           MZE54.4               1441      1  I21  1441  1441      1  SENS_2   
           MZE54.H               1441      1  I22  1441  1441      1  SENS_2   
           MZE54.K               1441      1  I21  1441  1441      1  SENS_2   
           SITE_01               1441      1  I23  1441  1441      1  SENS_2   
           SITE_03               1441      1  I23  1441  1441      1  SENS_2   
           SITE_05               1441      1  I23  1441  1441      1  SENS_2   
           SITE_08               1441      1  I22  1441  1441      1  SENS_2   
           SITE_09               1441      1  I22  1441  1441      1  SENS_2   
2          MZE54.H               1441      1  I12  1441  1441      1  SENS_1   
           SITE_03               1441      1  I11  1441  1441      1  SENS_1   
           SITE_05               1441      1  I11  1441  1441      1  SENS_1   
           SITE_08               1441      1  I12  1441  1441      1  SENS_1   
3          MZE54.H               1441      1  I22  1441  1441      1  SENS_2   
           SITE_01               1441      1  I23  1441  1441      1  SENS_2   
           SITE_03               1441      1  I23  1441  1441      1  SENS_2   
           SITE_05               1441      1  I23  1441  1441      1  SENS_2   
           SITE_08               1441      1  I22  1441  1441      1  SENS_2   
           SITE_09               1441      1  I22  1441  1441      1  SENS_2   
5          SITE_01               1441      1  I23  1441  1441      1  SENS_2   
           SITE_08               1441      1  I22  1441  1441      1  SENS_2   
           SITE_09               1441      1  I22  1441  1441      1  SENS_2   

                                
                          freq  
code_canal code_centrale        
0          MZE54.3        1441  
           MZE54.H        1441  
           SITE_03        1441  
           SITE_05        1441  
           SITE_08        1441  
1          MZE54.3        1441  
           MZE54.4        1441  
           MZE54.H        1441  
           MZE54.K        1441  
           SITE_01        1441  
           SITE_03        1441  
           SITE_05        1441  
           SITE_08        1441  
           SITE_09        1441  
2          MZE54.H        1441  
           SITE_03        1441  
           SITE_05        1441  
           SITE_08        1441  
3          MZE54.H        1441  
           SITE_01        1441  
           SITE_03        1441  
           SITE_05        1441  
           SITE_08        1441  
           SITE_09        1441  
5          SITE_01        1441  
           SITE_08        1441  
           SITE_09        1441